In [1]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
from input_data import load_data
from preprocessing import preprocess_graph, sparse_to_tuple, mask_test_edges
np.set_printoptions(threshold=np.inf)




W_gene      =  pd.read_table( "GBM_normalized/gene.txt", delim_whitespace=True).to_numpy()
W_methy     =  pd.read_table( "GBM_normalized/methy.txt", delim_whitespace=True).to_numpy()
W_mirna     =  pd.read_table( "GBM_normalized/mirna.txt", delim_whitespace=True).to_numpy()
W_truelabel =  pd.read_table( "GBM_normalized/truelabel.txt", delim_whitespace=True).to_numpy()
W_argsort =  pd.read_table( "GBM_normalized/argsort.txt", delim_whitespace=True).to_numpy()

gene      =  pd.read_table( "GBM/GLIO_Gene_Expression.txt", delim_whitespace=True).to_numpy().transpose()
methy     =  pd.read_table( "GBM/GLIO_Methy_Expression.txt", delim_whitespace=True).to_numpy().transpose()
mirna     =  pd.read_table( "GBM/GLIO_Mirna_Expression.txt", delim_whitespace=True).to_numpy().transpose()
truelabel =  pd.read_table( "GBM/GLIO_Survival.txt", delim_whitespace=True).to_numpy()

order = W_argsort.reshape(-1)-1

gene = gene[order]
methy = methy[order]
mirna = mirna[order]
truelabel = truelabel[order]

print(gene.shape)
print(methy.shape)
print(mirna.shape)

def get_nomralized_laplacian(a):
    b = np.diag(np.power(1.0 / a.sum(axis=1), 0.5))
    return (b @ a @ b)




# Some preprocessing
adj = W_gene
features = np.c_[mirna,methy]


# adj = W_methy
# features = np.c_[gene,mirna]


adj_norm = get_nomralized_laplacian(adj)
num_features = features.shape[1]

#pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
#norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

pos_weight = 1
norm = 1







(215, 12042)
(215, 1305)
(215, 534)


In [2]:
#Model definition Cell


adj_label_tensor = torch.Tensor(adj)
adj_norm_tensor  = torch.Tensor(adj_norm)
features_tensor  = torch.Tensor(features)

# from the previous cell following arrays are obtained : 
#adj_label
#adj_norm
#features
#val_edges_false
#test_edges
#test_edges_false




class Model(torch.nn.Module):

    def __init__(self,**kwargs):
        super(Model, self).__init__(**kwargs)
        
        
        
            

        self.W0 = torch.nn.Linear(num_features,6000)
        self.W1 = torch.nn.Linear(6000,3000)
        self.W2 = torch.nn.Linear(3000,1500)
        self.W3 = torch.nn.Linear(1500,700)
        self.W4 = torch.nn.Linear(700,1500)
        self.W5 = torch.nn.Linear(1500,3000)
        self.W6 = torch.nn.Linear(3000,6000)
        self.W7 = torch.nn.Linear(6000,num_features)

        
        param_list = []
        param_list += list(self.W0.parameters())
        param_list += list(self.W1.parameters())
        param_list += list(self.W2.parameters())
        param_list += list(self.W3.parameters())
        param_list += list(self.W4.parameters())
        param_list += list(self.W5.parameters())
        param_list += list(self.W6.parameters())
        param_list += list(self.W7.parameters())
        

        self.optimizer = torch.optim.Adam(param_list,lr=0.0001)
        self.recon = None
        
        
        

    


        
    def forward(self,x) : 
        self.layer_1 = torch.nn.functional.tanh(self.W0(x))
        self.layer_2 = torch.nn.functional.tanh(self.W1(self.layer_1))
        self.layer_3 = torch.nn.functional.tanh(self.W2(self.layer_2))
        self.layer_4 = torch.nn.functional.tanh(self.W3(self.layer_3))
        self.layer_5 = torch.nn.functional.tanh(self.W4(self.layer_4))
        self.layer_6 = torch.nn.functional.tanh(self.W5(self.layer_5))
        self.layer_7 = torch.nn.functional.tanh(self.W6(self.layer_6))
        self.layer_8 = self.W7(self.layer_7)

        self.x_prim = self.layer_8
        self.embedding = self.layer_4
        self.recon_loss = ((self.x_prim-x)**2).sum(1).mean(0)
        self.loss = self.recon_loss
        return self.loss.item()
        
    def backward(self) :
        
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        
        
    def get_cosine_sim_emb(self):
        #print(my_model.embedding)
        #my_model.forward(features_tensor)
        #my_model.recon = torch.sigmoid(my_model.embedding@my_model.embedding.transpose(0,1))
        emb = self.embedding.detach().numpy()

        norms =  (emb*emb).sum(1).reshape(-1,1)**0.5
        normalized_emb = emb/norms

        emb_matrix = np.abs( normalized_emb@normalized_emb.transpose())
        return emb_matrix
        

    def get_euclidian(self):
        emb = self.embedding.detach().numpy()
        
        emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
        for i in range(emb.shape[0]):
            for j in range(emb.shape[0]):
                emb_matrix[i,j] = ((emb[i]-emb[j])**2).sum()
                
        return emb_matrix
        
        



In [3]:
#Training Cell 

my_model = Model()


for epoch_counter in range(2000):
    
    epoch_loss = my_model.forward(features_tensor)
    my_model.backward()
    
    print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(epoch_loss))


print('\n',"#########################################################",'\n')






/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0001 train_loss= 1825.69080
Epoch: 0002 train_loss= 1808.21411
Epoch: 0003 train_loss= 1768.57642
Epoch: 0004 train_loss= 1699.01135
Epoch: 0005 train_loss= 1605.32117
Epoch: 0006 train_loss= 1525.61401
Epoch: 0007 train_loss= 1504.00989
Epoch: 0008 train_loss= 1460.81909
Epoch: 0009 train_loss= 1396.06384
Epoch: 0010 train_loss= 1365.73376
Epoch: 0011 train_loss= 1342.55762
Epoch: 0012 train_loss= 1325.07385
Epoch: 0013 train_loss= 1313.04321
Epoch: 0014 train_loss= 1289.04883
Epoch: 0015 train_loss= 1257.88367
Epoch: 0016 train_loss= 1232.99097
Epoch: 0017 train_loss= 1217.44678
Epoch: 0018 train_loss= 1202.16333
Epoch: 0019 train_loss= 1184.66174
Epoch: 0020 train_loss= 1171.78076
Epoch: 0021 train_loss= 1165.88623
Epoch: 0022 train_loss= 1158.93018
Epoch: 0023 train_loss= 1146.93994
Epoch: 0024 train_loss= 1131.49609
Epoch: 0025 train_loss= 1118.54236
Epoch: 0026 train_loss= 1109.91614
Epoch: 0027 train_loss= 1103.88770
Epoch: 0028 train_loss= 1099.31775
Epoch: 0029 train_lo

Epoch: 0242 train_loss= 20.64133
Epoch: 0243 train_loss= 19.45880
Epoch: 0244 train_loss= 18.33202
Epoch: 0245 train_loss= 17.25929
Epoch: 0246 train_loss= 16.23891
Epoch: 0247 train_loss= 15.26915
Epoch: 0248 train_loss= 14.34830
Epoch: 0249 train_loss= 13.47464
Epoch: 0250 train_loss= 12.64644
Epoch: 0251 train_loss= 11.86202
Epoch: 0252 train_loss= 11.11969
Epoch: 0253 train_loss= 10.41776
Epoch: 0254 train_loss= 9.75460
Epoch: 0255 train_loss= 9.12857
Epoch: 0256 train_loss= 8.53808
Epoch: 0257 train_loss= 7.98155
Epoch: 0258 train_loss= 7.45746
Epoch: 0259 train_loss= 6.96428
Epoch: 0260 train_loss= 6.50057
Epoch: 0261 train_loss= 6.06489
Epoch: 0262 train_loss= 5.65586
Epoch: 0263 train_loss= 5.27212
Epoch: 0264 train_loss= 4.91237
Epoch: 0265 train_loss= 4.57535
Epoch: 0266 train_loss= 4.25984
Epoch: 0267 train_loss= 3.96467
Epoch: 0268 train_loss= 3.68870
Epoch: 0269 train_loss= 3.43086
Epoch: 0270 train_loss= 3.19010
Epoch: 0271 train_loss= 2.96542
Epoch: 0272 train_loss= 2.75

KeyboardInterrupt: 

In [17]:
#my_model.forward(features_tensor)
emb = my_model.embedding.detach().numpy()
        
emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
for i in range(emb.shape[0]):
    for j in range(emb.shape[0]):
        #emb_matrix[i,j] =  ((emb[i]-emb[j])**2).sum()
        emb_matrix[i,j] =  np.exp(-((emb[i]-emb[j])**2).sum()/150)
                
print(emb_matrix)

[[1.         0.031005   0.03400177 0.04941661 0.02971807 0.02471345
  0.0414996  0.02499412 0.04409193 0.02296873 0.01863904 0.03707079
  0.02201405 0.03207657 0.01606899 0.0212626  0.03399362 0.04904897
  0.10913846 0.1257477  0.0208141  0.039887   0.04360557 0.03907165
  0.03198859 0.02070155 0.08466654 0.0325354  0.02803196 0.0290085
  0.02290596 0.02764902 0.01985426 0.01512266 0.02518477 0.0729161
  0.04626624 0.03689488 0.06217452 0.02395912 0.0233228  0.04119815
  0.0360862  0.01537368 0.04738748 0.07737171 0.02418204 0.06225829
  0.08275772 0.10032555 0.05162792 0.03705093 0.0201643  0.07037047
  0.02423013 0.04986117 0.02092968 0.0337117  0.07826074 0.02493399
  0.03130129 0.06700817 0.03498643 0.02274371 0.02478077 0.03314075
  0.04104516 0.01272881 0.01650341 0.0180065  0.01282727 0.01864534
  0.0238141  0.15642051 0.01356843 0.03064755 0.1037902  0.0966071
  0.02829923 0.02395494 0.046422   0.03001969 0.03959912 0.03281005
  0.07837662 0.03476782 0.13701847 0.04756634 0.017

In [18]:
np.savetxt("gvae_fusion_results/emb_150.txt",emb,delimiter='\t')
np.savetxt("gvae_fusion_results/res_150.txt",emb_matrix,delimiter='\t')

In [19]:
from sklearn.cluster import KMeans
X = emb
n_clusters=3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

for i in range(n_clusters):
    indx = np.argwhere(kmeans.labels_==i).reshape(-1)
    print(truelabel[indx][:,1].mean())



715.4767441860465
584.5172413793103
478.98


In [21]:
emb_matrix = my_model.get_cosine_sim_emb()
print(emb_matrix)
np.savetxt("gvae_fusion_results/res_cosine.txt",emb_matrix,delimiter='\t')

[[9.99999762e-01 1.07528433e-01 2.69530844e-02 9.13520753e-02
  1.03215739e-01 7.80488998e-02 5.88160381e-02 2.92040147e-02
  3.22823524e-02 2.40955632e-02 2.25115180e-01 5.83793297e-02
  6.07349798e-02 7.87358359e-02 9.43368226e-02 6.90505430e-02
  2.34133713e-02 4.30251881e-02 3.36033434e-01 3.34922850e-01
  1.67553216e-01 1.98978633e-02 2.84192488e-02 2.92781666e-02
  1.31397210e-02 3.53594646e-02 2.65772641e-01 4.52555791e-02
  4.13791835e-02 2.36687094e-01 9.38652232e-02 1.55723453e-01
  2.88900912e-01 2.28089005e-01 5.03865629e-03 2.44999051e-01
  1.20611086e-01 2.92277858e-02 1.25792027e-01 4.08527851e-02
  2.19282031e-01 1.06328651e-01 9.04962942e-02 2.38926888e-01
  1.54283881e-01 2.10222602e-01 1.37868032e-01 2.08886683e-01
  2.64776915e-01 2.95316100e-01 6.46592751e-02 5.15146069e-02
  4.34365980e-02 1.80428043e-01 1.37596533e-01 1.18417107e-01
  1.36523679e-01 8.89368914e-03 1.69216335e-01 1.62840560e-02
  3.34962867e-02 1.38020709e-01 1.60055198e-02 5.96024096e-02
  1.8452